**Simulating Survivability**

Analyzing the time it takes to reach the patient for the vector and the responders

In [1]:
import pandas as pd
import numpy as np
import openrouteservice
from openrouteservice import client
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import time
import geopy.distance
from Simulation_Class import simulation

Upload intervention data

In [2]:
intervention = pd.read_csv("Data/interventions.csv")
intervention.rename(columns = {'Longitude intervention':'longitude', 'Latitude intervention':'latitude'}, inplace = True)
intervention['coordinates'] = list(zip(intervention['longitude'], intervention['latitude']))

Uploading simulated responder data for 0,5% and 1% of population as first responders

In [3]:
responder_one = pd.read_csv("Data/1%_FRs.csv")

In [4]:
responder_one.rename(columns = {'x':'longitude', 'y':'latitude'}, inplace = True)
responder_one['coordinates'] = list(zip(responder_one['longitude'], responder_one['latitude']))

In [5]:
responder_half = pd.read_csv("Data/0.5%_FRs.csv")

In [6]:
responder_half.rename(columns = {'x':'longitude', 'y':'latitude'}, inplace = True)
responder_half['coordinates'] = list(zip(responder_half['longitude'], responder_half['latitude']))

Uploading AED location

In [7]:
AED_location = pd.read_csv("Data/filtered_AED_loc.csv")

Combining the Ambulance, PIT, and MUG destination to find all possible Vector locations.

This can be done as in a cardiac arrest case all units could possibly reply to a call.

In [8]:
amb_loc = pd.read_parquet("Data/ambulance_locations.parquet.gzip")
Ambulances = amb_loc[amb_loc['province']=='Vlaams-Brabant']
# Only keep rows that contain 30 as all postla code of Leuven start with 30
Ambulances = Ambulances[Ambulances['departure_location'].str.contains('30')]
Ambulances = Ambulances.reset_index(drop = True)
df = pd.DataFrame()
df['Name'] = Ambulances['departure_location_number']
df['longitude'] = Ambulances['longitude']
df['latitude'] = Ambulances['latitude']

In [9]:
pit_loc = pd.read_parquet("Data/pit_locations.parquet.gzip")
Pit = pit_loc[pit_loc['province']=='Vlaams Brabant/Brabant flamand']
Pit = Pit.reset_index(drop = True)
Pit = Pit[Pit['ambucode']==288]
Pit = Pit.reset_index(drop = True)
df_2 = pd.DataFrame()
df_2['Name'] = Pit['unit_id']
df_2['longitude'] = 4.6716518
df_2['latitude'] = 50.8791702

In [10]:
mug_loc = pd.read_parquet("Data/mug_locations.parquet.gzip")
Mug = mug_loc[mug_loc['province']=='Vlaams-Brabant']
Mug = Mug[Mug['postal_code']==3000]
Mug = Mug.reset_index(drop = True)

# Googled the latitude and longitude
df_3 = pd.DataFrame()
df_3['Name'] = Mug['hospital_id']
df_3['longitude'] = 4.6690603
df_3['latitude'] = 50.8784361

In [11]:
df_vectors = pd.concat([df, df_2, df_3], ignore_index = True)
df_vectors['coordinates'] =  list(zip(df_vectors.loc[:,'longitude'], df_vectors.loc[:,'latitude']))
Vectors = df_vectors[['longitude', 'latitude','coordinates']]

Initialize the simulation class

In [12]:
simulate = simulation()

In [13]:
simulate

We will continue by iterating through the former intervention cases to find the time the responders would need. We also find the time the Vector would need if they are dispatched right away.

As the API can only be called 2000 times per day the simulation has to be done over the course of multiple days as finidng optimal routes takes around 60 
calls per iteration.

The output will be stored in a csv file and will be combined as soon as all iterations are complete

In [14]:
len(intervention)

81

In [15]:
intervention.loc[0]

T1                2022-06-07 07:02:17
T3                2022-06-07 07:11:24
latitude                    50.861866
longitude                     4.70821
Response Time                   547.0
coordinates      (4.70821, 50.861866)
Name: 0, dtype: object

**Simulation when 1% of the population are first responders**

In [16]:
#first iteration
df_final = pd.DataFrame(columns = ['Patient_loc', 'Responder_lon', 'Responder_lat', 
        'duration_Responder', 'AED_lon', 'AED_lat','duration_AED',
        'Vector_lon', 'Vector_lat', 'duration_Vector'])
for i in range(30):
    print(i)
    df = simulate.fastest_time(intervention.loc[i], responder_one, AED_location, Vectors)
    df_final = pd.concat([df_final,df])
    df_final = df_final.reset_index(drop = True)

0
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
1


C:\Users\leonw\AppData\Local\Temp\ipykernel_25140\2267795599.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final,df])


Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
2
No vector is close by. Increase of thresholds
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
3
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
4
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
5
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
6
Comparing direct responder vs. vectors
Duration for Responders found
Duration for Vectors found
7
Comparing resopnders vs. vectors
Duration for respo

In [17]:
df_final

,Patient_loc,Responder_lon,Responder_lat,duration_Responder,AED_lon,AED_lat,duration_AED,Vector_lon,Vector_lat,duration_Vector
0,"(4.70821, 50.861866)",4.709184,50.862418,67.3,4.709673,50.862839,516.8,4.699833,50.872354,339.2
1,"(4.716699, 50.873174)",4.716120,50.874038,127.6,4.715005,50.872849,322.5,4.724344,50.869655,258.4
2,"(4.703247, 50.92814)",4.702441,50.927772,57.5,4.701941,50.925745,304.7,4.665862,50.877460,703.8
3,"(4.730246, 50.87455)",4.731371,50.874638,56.5,4.728077,50.874903,542.1,4.724344,50.869655,475.5
4,"(4.696672, 50.879098)",4.695131,50.878730,98.9,4.698429,50.879273,358.2,4.695398,50.878535,35.7
5,"(4.705947, 50.886687)",4.706797,50.885970,99.0,4.704742,50.885932,229.0,4.699833,50.872354,511.8
6,"(4.690584, 50.89003)",4.694123,50.891359,310.4,No AED,No AED,No AED,4.673199,50.878079,327.8
7,"(4.692755, 50.865466)",4.693164,50.864972,104.9,4.693832,50.864072,718.1,4.699833,50.872354,198.7
8,"(4.700683, 50.87929)",4.700665,50.879306,1.3,4.700471,50.879172,39.0,4.695398,50.878535,641.1
9,"(4.681497, 50.854151)",4.680437,50.854875,100.5,No AED,No AED,No AED,4.699833,50.872354,470.7


In [18]:
# Save the file path
file_path = "Data/simulation_one_1st.csv"
# Save the dataframe to a CSV file
df_final.to_csv(file_path, index=False)

The second iteration is conducted

In [19]:
#second iteration
df_final = pd.DataFrame(columns = ['Patient_loc', 'Responder_lon', 'Responder_lat', 
        'duration_Responder', 'AED_lon', 'AED_lat','duration_AED',
        'Vector_lon', 'Vector_lat', 'duration_Vector'])
for i in range(30,60):
    print(i)
    df = simulate.fastest_time(intervention.loc[i], responder_one, AED_location, Vectors)
    df_final = pd.concat([df_final,df])
    df_final = df_final.reset_index(drop = True)

30
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are the same
Duration for Vectors found
31


C:\Users\leonw\AppData\Local\Temp\ipykernel_25140\2280843813.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final,df])


Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
32
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
33
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are the same
Duration for Vectors found
34
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are the same
Duration for Vectors found
35
Comparing direct responder vs. vectors
Duration for Responders found
Duration for Vectors found
36
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
37
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Dire

In [20]:
df_final

,Patient_loc,Responder_lon,Responder_lat,duration_Responder,AED_lon,AED_lat,duration_AED,Vector_lon,Vector_lat,duration_Vector
0,"(4.700399, 50.923948)",4.701941,50.925745,198.3,4.700618,50.926063,303.4,4.665862,50.877460,622.0
1,"(4.71539, 50.88122)",4.713584,50.881932,179.6,4.713036,50.881074,224.4,4.699833,50.872354,353.9
2,"(4.71745, 50.874271)",4.718112,50.874427,27.6,4.716658,50.874547,457.7,4.724344,50.869655,337.3
3,"(4.69952, 50.866911)",4.699123,50.867396,56.9,4.700474,50.867696,1020.0,4.699833,50.872354,327.8
4,"(4.711828, 50.871651)",4.712238,50.872168,45.4,4.713172,50.872107,494.7,4.699833,50.872354,275.6
5,"(4.730961, 50.850728)",4.725530,50.849836,342.4,No AED,No AED,No AED,4.724344,50.869655,385.2
6,"(4.663558, 50.876016)",4.663760,50.877148,198.0,4.6598,50.875534,284.0,4.665862,50.877460,479.4
7,"(4.735979, 50.888449)",4.735845,50.888155,13.4,4.735199,50.888172,749.6,4.724344,50.869655,645.1
8,"(4.728948, 50.868191)",4.730575,50.867953,82.6,No AED,No AED,No AED,4.724344,50.869655,88.3
9,"(4.698691, 50.865606)",4.697731,50.866092,14.4,4.698232,50.864543,938.6,4.699833,50.872354,206.7


In [21]:
# Save the file path
file_path = "Data/simulation_one_2nd.csv"
# Save the dataframe to a CSV file
df_final.to_csv(file_path, index=False)

In [22]:
#third iteration
df_final = pd.DataFrame(columns = ['Patient_loc', 'Responder_lon', 'Responder_lat', 
        'duration_Responder', 'AED_lon', 'AED_lat','duration_AED',
        'Vector_lon', 'Vector_lat', 'duration_Vector'])
for i in range(60,len(intervention)):
    print(i)
    df = simulate.fastest_time(intervention.loc[i], responder_one, AED_location, Vectors)
    df_final = pd.concat([df_final,df])
    df_final = df_final.reset_index(drop = True)

60
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
61


C:\Users\leonw\AppData\Local\Temp\ipykernel_25140\4025076053.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final,df])


Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
62
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are the same
Duration for Vectors found
63
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
64
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
65
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
66
Comparing direct responder vs. vectors
Duration for Responders found
Duration for Vectors found
67
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest 

C:\Users\leonw\anaconda3\Lib\site-packages\openrouteservice\client.py:195: UserWarning: Server down.
Retrying for the 1st time.
  warnings.warn('Server down.\nRetrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\leonw\anaconda3\Lib\site-packages\openrouteservice\client.py:195: UserWarning: Server down.
Retrying for the 2nd time.
  warnings.warn('Server down.\nRetrying for the {0}{1} time.'.format(retry_counter + 1,


Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
74
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
75
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are the same
Duration for Vectors found
76
Comparing direct responder vs. vectors
Duration for Responders found
Duration for Vectors found
77
Comparing direct responder vs. vectors
Duration for Responders found
Duration for Vectors found
78
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
79
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
80
Comparing direct responder vs. vectors
Duration for Respond

In [23]:
df_final

,Patient_loc,Responder_lon,Responder_lat,duration_Responder,AED_lon,AED_lat,duration_AED,Vector_lon,Vector_lat,duration_Vector
0,"(4.695989, 50.865144)",4.696955,50.866943,161.8,4.695789,50.867158,636.8,4.699833,50.872354,187.6
1,"(4.726644, 50.873674)",4.726799,50.873856,14.8,4.726127,50.873491,179.8,4.724344,50.869655,356.1
2,"(4.72935, 50.900387)",4.729099,50.900987,65.1,4.729871,50.901447,5000.0,4.665862,50.877460,563.4
3,"(4.690636, 50.876538)",4.689608,50.876599,61.8,4.692353,50.876531,1196.5,4.699833,50.872354,256.3
4,"(4.704618, 50.887542)",4.703671,50.888129,112.4,4.704742,50.885932,368.4,4.673199,50.878079,547.8
5,"(4.722256, 50.880999)",4.721091,50.881465,82.2,4.721773,50.882072,706.3,4.724344,50.869655,507.5
6,"(4.73756, 50.862344)",4.737364,50.861318,52.9,No AED,No AED,No AED,4.724344,50.869655,374.6
7,"(4.698691, 50.865606)",4.697731,50.866092,14.4,4.698232,50.864543,938.6,4.699833,50.872354,206.7
8,"(4.722139, 50.86866)",4.722013,50.869474,53.2,No AED,No AED,No AED,4.724344,50.869655,98.5
9,"(4.736791, 50.863533)",4.736850,50.863480,4.1,4.735141,50.862816,5000,4.724344,50.869655,366.7


In [24]:
# Save the file path
file_path = "Data/simulation_one_3rd.csv"
# Save the dataframe to a CSV file
df_final.to_csv(file_path, index=False)

**Simulation with 0.5% of the population as responders**

In [ ]:
#first iteration
df_final = pd.DataFrame(columns = ['Patient_loc', 'Responder_lon', 'Responder_lat', 
        'duration_Responder', 'AED_lon', 'AED_lat','duration_AED',
        'Vector_lon', 'Vector_lat', 'duration_Vector'])
for i in range(30):
    print(i)
    df = simulate.fastest_time(intervention.loc[i], responder_half, AED_location, Vectors)
    df_final = pd.concat([df_final,df])
    df_final = df_final.reset_index(drop = True)

0
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
1


C:\Users\leonw\AppData\Local\Temp\ipykernel_41916\2792188279.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final,df])


Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
2
No vector is close by. Increase of thresholds
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
3
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
4
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
5
Comparing resopnders vs. vectors
Duration for responders found
Duration for AEDs found
Fastest Direct and Indirect are not the same
Duration for Vectors found
6
Comparing direct responder vs. vectors
Duration for Responders found
Duration for Vectors found
7
Comparing resopnders vs. vectors
Duration for respo

In [ ]:
# Save the file path
file_path = "Data/simulation_half_1st.csv"
# Save the dataframe to a CSV file
df_final.to_csv(file_path, index=False)

In [ ]:
#second iteration
df_final = pd.DataFrame(columns = ['Patient_loc', 'Responder_lon', 'Responder_lat', 
        'duration_Responder', 'AED_lon', 'AED_lat','duration_AED',
        'Vector_lon', 'Vector_lat', 'duration_Vector'])
for i in range(30):
    print(i)
    df = simulate.fastest_time(intervention.loc[i], responder_half, AED_location, Vectors)
    df_final = pd.concat([df_final,df])
    df_final = df_final.reset_index(drop = True)

In [ ]:
# Save the file path
file_path = "Data/simulation_half_1st.csv"
# Save the dataframe to a CSV file
df_final.to_csv(file_path, index=False)